# Subset Arithmetic in $\mathcal{P}_{\textrm{fin},0}(\mathbb{N})$: A Walkthrough

In this notebook are some examples and explanations of functions I wrote to help with my dissertation research.  The dissertation itself lends more time to theoretical matters and to proofs; here, I would like to focus instead on the computational tools that helped me formulate conjectures that eventually became the main results of my thesis (and maybe some other less-than-resolved conjectures).  Also, since this flavor of mathematics does not require extensive background, I see this as an opportunity to present some combinatorial factorization theory to any curious individuals who may not identify as mathematicians.

Power Monoids were first introduced to the mathematical literature by Yushuang Fan and Salvo Tringali in their 2018 article [Power Monoids: A Bridge Between Factorization Theory and Arithmetic Combinatorics](https://arxiv.org/abs/1701.09152).  Further theoretical discussion of power monoids can be found in [my joint paper with Salvo](https://arxiv.org/abs/1804.10913), which makes up the first couple chapters of my [dissertation](https://github.com/austinant/thesis/blob/master/AAA%20Dissertation.pdf).


While there is a great deal of theoretical discussion to be had over power monoids in general, this notebook is intended to demonstrate some of the functions I've written specifically for doing arithmetic in the Natural Power Monoid $\mathcal{P}_{\textrm{fin}}(\mathbb{N})$; that is, the collection of non-empty subsets of the natural numbers $\mathbb{N} = \{0,1,2,3,\dots \}$.
This collection of subsets inherits the addition $+$ from the natural numbers in the following way:

$$ X+Y := \{x+y: x\in X, y\in Y \} $$

Essentially, the sum $X+Y$ records (without duplication or care for order) every possible sum of pairs from $X$ and $Y$.  Here's an example calculation:

\begin{align*}
\{0,1,3\} + \{0,3\} 
&= \{0+0,0+3,1+0,1+3,3+0,3+3\} \\
&= \{0,3,1,4,3,6\} \\
&= \{0,1,3,4,6\}
\end{align*}

For some more example calculations, import the PfinN module and run the following couple of cells.

In [1]:
import PfinN as pn

In [2]:
X = {0,1,3}
Y = {2,5}
pn.setsum(X,Y)

{2, 3, 5, 6, 8}

Just like with $+$ for integers, we may add more than two sets at a time:

In [3]:
X = {0,2}
Y = {0,4}
Z = {0,8}
pn.setsum(X,Y,Z)

{0, 2, 4, 6, 8, 10, 12, 14}

Great; we can add.  Now what?

One might wonder if we can subtract sets. This is actually impossible in a certain sense.  If $X + Y = Z$, then we'd like to say that $Y = Z - X$.  However, there may be multiple choices of $Y$ (while keeping $X$ and $Z$ the same) which make the first equation true.  For example:

In [9]:
#X = {0,1} and Y = {0,2}:
pn.setsum({0,1},{0,2})

{0, 1, 2, 3}

In [10]:
# Change to Y = {0,1,2}, keep X the same; get the same sum X+Y:
pn.setsum({0,1},{0,1,2})

{0, 1, 2, 3}

## Saturated Cofactors

What turns out to be true: if $X+Y = Z$, there are not unique $X$ and $Y$ which make this equation true, but if we fix $X$ and $Z$, there is a unique "largest" possible $Y$.  This $Y$, called the *saturated cofactor* of $X$ in $Z$, is determined by $Y = Z\!:\!X$, where

$$Z\!:\!X = \bigcap_{x\in X} (Z-x)$$

Let's see an example calculation of a saturated cofactor.  First, we'll do a small one "by hand". We saw above that $\{0,1\} + \{0,2\} = \{0,1,2,3\}$.  Taking $X = \{0,1\}$ and $Z = \{0,1,2,3\}$, we calculate that

\begin{align*}
\{0,1,2,3\}\!:\!\{0,1\} 
&= (\{0,1,2,3\}-0) \cap (\{0,1,2,3\} - 1) \\
&= \{0,1,2,3\} \cap \{-1,0,1,2\} \\
&= \{0,1,2\}
\end{align*}

We also know from above that this new set also works as a $Y$: $\{0,1\} + \{0,1,2\} = \{0,1,2,3\}$.

Now we'll have the "cofactor" function from PfinN work out a larger example.

In [11]:
X = {0,1,3,4,6,7,9}
Z = set(range(20))-{14,16} #this is the set {0,1,2,3,4,5,6,7,8,9,10,11,12,13,15,17,18,19} (missing 14,16)
pn.cofactor(Z,X)

{0, 1, 2, 3, 4, 6}

### Summary: Facts about saturated cofactors

1. $Z\!:\!X$ is the largest possible choice of $Y$ such that $X + Y = Z$ (any $Y$ must be a subset of $Z\!:\!X$)


2. If $X + Z\!:\!X \neq Z$ then there is *NO* possible $Y$ such that $X+Y=Z$

We will take advantage of both of these facts as we move into our discussion of factorization.

## Factorization

The equations $X + Y = Z$ and $Z = X + Y$ have the same literal meaning: when $X$ and $Y$ are added together, the result is $Z$.  However, viewing equations as sentences, those two equations have different subjects/objects, which lends different interpretations to each:

In $X + Y = Z$, the subjects (or, if you like, inputs) are $X$ and $Y$, and the object (or output) is $Z$.  This sentence answers the question "What happens when we add $X$ and $Y$?"

In $Z = X + Y$, we have essentially reversed the subject/object (or input/output) roles, answering a different question: "Where does $Z$ come from?" or "Which things can be put together to get $Z$?"

*Factorization theory* is the study of answering questions of the latter type, not just in our setting involving subsets of the natural numbers, but in any setting where one has an operation (like $+$ or $\times$) for combining objects (making this a rather huge area of study).  However, we will remain restricted to our setting for simplicity.

### Atoms

If I want to ask a question like "Which things can be put together to get $Z$?" I need an idea of which things I'm allowed to put together.  If we find that $Z = X + Y$, but then $X = U+V$, and $U = ...$, this could lead to a complicated and unclear answer to our original question.  So we should try decomposing $Z$ into parts which can't be decomposed further; these will be called atoms.

An *atom* is a nontrivial set $A$ which cannot be decomposed into any nontrivial subsets, meaning that if we try to write $A = X + Y$, then we must either choose $X = \{0\}$ or $Y = \{0\}$ ("nontrivial" just means "not equal to $\{0\}$").

A *factorization* of some nontrivial set $X$ is a decomposition into atoms $A_1,\dots,A_k$: $X = A_1 + \cdots + A_k$.  In our setting of subsets of the natural numbers, a factorization may not be unique; there may also be atoms $B_1,\dots, B_\ell$ such that $X = B_1 + \cdots + B_\ell$. (this non-uniqueness is actually most of the fun!)

Let's see some examples: you might try by hand to find all of the factorizations of the interval $[0,5] = \{0,1,2,3,4,5\}$.  There aren't excessively many of them, but finding a systematic, catch-all strategy is perhaps a bit of a trick.  Try thinking about it, then you can run the following cell to check your work.

In [12]:
X = {0,1,2,3,4,5}
for f in pn.facs_by_force(X):
    print(f)
#Each of the results lists atoms which could be added to get {0,1,2,3,4,5}, 
# though they are returned as tuples for the sake of convenience within the code

((0, 1), (0, 2), (0, 2))
((0, 1), (0, 1), (0, 1), (0, 1), (0, 1))
((0, 1), (0, 1), (0, 2, 3))
((0, 1), (0, 2, 3, 4))
((0, 1), (0, 1), (0, 3))
((0, 1), (0, 1), (0, 1, 3))
((0, 1), (0, 1, 2, 4))
((0, 1), (0, 1), (0, 1), (0, 2))


If we try running the code above to factor $X = [0,10]$ or $X = [0,15]$, we won't see a result for a long time.  Each of these sets has many factorizations.  In general, larger sets can have excessively many factorizations (with intervals being the worst offenders in a quantifiable sense to be discussed later).  Herein lies the computational challenge in investigating the factorization behavior of subsets of $\mathbb{N}$.

Run the cell below to see how many factorizations some sets have (it'll take some time).  Notice we're not using the same function as before to find all the factorizations; we'll talk about how we arrived at this function later.

In [15]:
for n in range(6,18):
    print('[0,'+str(n)+'] has '+str(len(pn.facs_by_type(set(range(n+1)))))+' factorizations ')

[0,6] has 16 factorizations 
[0,7] has 37 factorizations 
[0,8] has 81 factorizations 
[0,9] has 183 factorizations 
[0,10] has 424 factorizations 
[0,11] has 986 factorizations 
[0,12] has 2288 factorizations 
[0,13] has 5316 factorizations 
[0,14] has 12374 factorizations 
[0,15] has 28792 factorizations 
[0,16] has 66784 factorizations 
[0,17] has 154688 factorizations 
